In [68]:
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os
import numpy as np

pic_size = 48                         


base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

# Function to preprocess images
def preprocess_image(img_path, target_size=(48, 48)):
    """Load and preprocess an image."""
    img = load_img(img_path, target_size=target_size, color_mode="grayscale")  # Grayscale for custom CNNs
    img_array = img_to_array(img)  # Convert to NumPy array
    img_array /= 255.0  # Normalize pixel values
    return img_array

# Function to load and preprocess dataset
def load_dataset(base_path, subset='train'):
    """Load and preprocess the dataset."""
    images = []
    labels = []
    label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}

    subset_path = os.path.join(base_path, subset)
    for expression in os.listdir(subset_path):
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue  

        for img_name in os.listdir(expression_path):
            img_path = os.path.join(expression_path, img_name)
            try:
                img_array = preprocess_image(img_path)
                images.append(img_array)
                labels.append(label_map[expression])  
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")

    return np.array(images), np.array(labels)



# Function to visualize sample images
def visualize_samples(base_path, subset='train'):
    """Visualize sample images for each expression."""
    plt.figure(figsize=(12, 20))           
    counter = 0                            
    subset_path = os.path.join(base_path, subset)
    
    for expression in os.listdir(subset_path):                      
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue                                                
        
        for i, img_name in enumerate(os.listdir(expression_path)[:5]):  
            counter += 1                                               
            plt.subplot(7, 5, counter)                                  
            img_path = os.path.join(expression_path, img_name)
            img_array = preprocess_image(img_path)
            plt.imshow(img_array, cmap="gray")                         
            plt.xlabel(expression)
    
    plt.tight_layout()
    plt.show()


# Function to count images per expression
def count_images_per_expression(base_path, subset='train'):
    """Count and display the number of images per expression."""
    subset_path = os.path.join(base_path, subset)
    for expression in os.listdir(subset_path):
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue
        
        num_images = len(os.listdir(expression_path))
        print(f"{num_images} {expression} images")


In [ ]:
count_images_per_expression(base_path, subset='train')
##visualize_samples(base_path, subset='train')

In [ ]:
count_images_per_expression(base_path, subset='validation')
#visualize_samples(base_path, subset='validation')

In [71]:
#### Error handling:---- don't run now
#for img_name in os.listdir(expression_path):
   # img_path = os.path.join(expression_path, img_name)
   # try:
    #    img_array = preprocess_image(img_path)
     #   images.append(img_array)
     #   labels.append(label_map[expression])
   # except Exception as e:
   #     print(f"Error loading image {img_name}: {e}")


-----------------------------------------------------------------------------------

In [82]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Size of the image: 48x48 pixels
pic_size = 48                                                       


# Input path for the images
base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

batch_size = 64   

train_datagen = ImageDataGenerator(
    rescale=1.0/255,             
    rotation_range=10,            
    width_shift_range=0.1,        
    height_shift_range=0.1,               
    zoom_range=0.1,               
    horizontal_flip=True,        
    fill_mode='nearest'           
)

      
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:

train_generator = train_datagen.flow_from_directory(base_path + "/train",           
                                                    target_size=(pic_size,pic_size), 
                                                    color_mode="grayscale",           
                                                    batch_size=batch_size,            
                                                    class_mode='categorical',         
                                                    shuffle=True)                    
validation_generator = validation_datagen.flow_from_directory(base_path + "/validation",
                                                    target_size=(pic_size,pic_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

-----------------------------------------------------------------------------------

In [99]:
class_weight_dict = {0: 1.0311, 1: 9.4433, 2: 1.0035, 3: 0.5747, 4: 0.8264, 5: 0.8338, 6: 1.2846}

class_weight_dict = {i: weight for i, weight in enumerate([
    1.031125898894494, 9.443315858453474, 1.0034817729187702,
    0.5747188322565207, 0.8264322991340254, 0.8337962159347335,
    1.2846445286382884
])}


In [86]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

# Number of labels :
num_labels =  7

In [88]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GlobalAveragePooling2D

model = Sequential()
# 1st Convolution layer
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Dropout(0.25)) 

# 2nd Convolution layer
model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  

In [89]:
opt = Adam(learning_rate=0.0001)                                                                 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])   

In [ ]:
model.summary()

In [ ]:
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size
print(f"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(
    "model_weights.keras",
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
callbacks=[checkpoint, early_stop]


steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size
print(f"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")

history = model.fit(
    x=train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight=class_weight_dict,
    callbacks=[checkpoint, early_stop]
)

In [ ]:
print(f"Validation samples: {validation_generator.n}")
print(f"Batch size: {validation_generator.batch_size}")
print(f"Steps per epoch (validation): {len(validation_generator)}")


In [ ]:
print(class_weight_dict)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))

# Plot Loss
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer: Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')

plt.show()


In [ ]:
# Show the confusion matrix of our predictions

# Compute predictions
predictions = model.predict(validation_generator)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = validation_generator.classes
class_names = list(validation_generator.class_indices.keys())

from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()


In [ ]:

model.save('/Users/pratiksha/Documents/Pratiksha/Documents/GitHub/GitHub/Face-expression-recognition-with-Deep-Learning/my_model.keras')


he 450/450 in the training log represents the number of steps (batches) that have been completed during the current epoch. It shows the following:

450/450 means that the model has processed all 450 steps (batches) of the current epoch.
The first 450 represents the number of steps that have been completed so far in the epoch.
The second 450 is the total number of steps that the model will go through in this epoch (i.e., the number of batches).
Since the model has reached 450/450, it means the epoch has finished, and all training data has been processed once.

In short, it indicates the current progress of training in terms of the batches completed in the current epoch

- **Batch size** refers to the number of samples the model processes before updating its weights.
- The **steps per epoch** refers to how many batches are needed to complete one full pass through the training dataset.

So, in your case, the training generator is set to:

- **Batch size = 64**: This means each batch contains 64 images.
- **Steps per epoch = 450**: This means that in each epoch, the model will process 450 batches (each containing 64 images). This results in processing a total of \( 450 \times 64 \) images in one epoch.

If we calculate the total number of images processed during one epoch:

\[
\text{Total images per epoch} = 450 \times 64 = 28,800 \text{ images}
\]

This means, in each epoch, the model will go through 28,800 images (divided into 450 batches of 64 images each).

So, when you see `450/450`, it simply means that all 450 batches (of 64 images each) have been processed for that epoch.

the value `450` for the **steps per epoch** is typically determined based on the size of your dataset and the batch size you are using.

If you haven't explicitly set it, then the number `450` is calculated automatically based on the data generator and batch size.

### How `steps_per_epoch` is calculated:
It is determined as:
\[
\text{steps per epoch} = \frac{\text{total number of samples in training set}}{\text{batch size}}
\]

For instance:
- If your training dataset has 28,800 images, and your batch size is 64, then:
\[
\text{steps per epoch} = \frac{28,800}{64} = 450
\]

So, `450` is not something you manually set, but it's the result of this calculation based on your data size and batch size.